In [16]:
import pandas as pd
print(f"Versão do Pandas: {pd.__version__}")

import pyspark as pyspark
print(f"Versão do LightGBM: {pyspark.__version__}")

Versão do Pandas: 2.1.4
Versão do LightGBM: 4.0.0


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import lightgbm as lgb

In [2]:
final_dataset = pd.read_parquet("../data/processed/final_data_numeric.parquet")

A divisão dos dados é feita com base nos account_ids, garantindo que as ofertas de um mesmo cliente não apareçam em conjuntos diferentes (treino, validação e teste). Isso simula um cenário real, onde o modelo seria treinado com um grupo de clientes e validado/testado com clientes novos, evitando o vazamento de dados (data leakage).

Neste case, a única feature considerada é a offer_id, pois ela já representa uma combinação única das características de cada oferta. Essa abordagem simplifica uso das variáveis e permite avaliar o impacto de cada oferta como um todo, mas também limita a capacidade do modelo de generalizar para novas combinações de ofertas, já que o modelo não vê separadamente os atributos que compõem cada offer.

In [3]:
# --- 1. Dividir account_ids em treino (60%), validação (20%), teste (20%) ---
account_ids = final_dataset['account_id'].unique()

# Primeiro split treino vs temp (40% para temp)
train_ids, temp_ids = train_test_split(account_ids, test_size=0.3, random_state=42)

# Depois split temp em validação, teste (20%, 20%)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

print(f'Treino: {len(train_ids)}, Validação: {len(val_ids)}, Teste: {len(test_ids)}')

# --- 2. Criar datasets filtrados ---
train_set = final_dataset[final_dataset['account_id'].isin(train_ids)].copy()
val_set = final_dataset[final_dataset['account_id'].isin(val_ids)].copy()
test_set = final_dataset[final_dataset['account_id'].isin(test_ids)].copy()

# --- 3. Pré-processamento ---
categorical_features = ['offer_id']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Treino
X_train = train_set[categorical_features]
y_train = train_set['has_offer_completed']
X_train_proc = preprocessor.fit_transform(X_train)

# Validação
X_val = val_set[categorical_features]
y_val = val_set['has_offer_completed']
X_val_proc = preprocessor.transform(X_val)

# Teste
X_test = test_set[categorical_features]
y_test = test_set['has_offer_completed']
X_test_proc = preprocessor.transform(X_test)

# --- 4. Criar grupos para ranking ---
group_train = train_set.groupby('account_id').size().to_numpy()
group_val = val_set.groupby('account_id').size().to_numpy()
group_test = test_set.groupby('account_id').size().to_numpy()

# --- 5. Dataset LightGBM ---
lgb_train = lgb.Dataset(X_train_proc, label=y_train, group=group_train)
lgb_val = lgb.Dataset(X_val_proc, label=y_val, group=group_val)

# --- 6. Treinar Ranker ---
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3],
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_val],
    num_boost_round=200
)

Treino: 10187, Validação: 2183, Teste: 2183


## Inferência e Geração de Recomendações Top-3

Após o treinamento, usamos o modelo para prever scores para todas as ofertas do conjunto de teste. Para simular um cenário real de recomendação, criamos todas as combinações possíveis de clientes e ofertas. O modelo então atribui um score a cada combinação. Com base nesses scores, selecionamos as top-3 ofertas para cada cliente, que representam as recomendações mais promissoras.

In [4]:
test_set['score'] = model.predict(X_test_proc)

In [5]:
# 1. Obter a lista única de clientes e ofertas
all_offers = final_dataset['offer_id'].unique().copy()
test_clients = test_set['account_id'].unique().copy()

# 2. Criar um DataFrame com todas as combinações cliente-oferta
all_test_combinations = pd.DataFrame({
    'account_id': np.repeat(test_clients, len(all_offers)),
    'offer_id': np.tile(all_offers, len(test_clients))
})

# 3. Adicionar as ofertas de cada cliente
# Mesclar as ofertas de `test_set` com as novas combinações. 
client_features = test_set[['account_id']].drop_duplicates()
all_test_combinations = pd.merge(all_test_combinations, client_features, on='account_id', how='left')

# 4. Pré-processar os novos dados
X_all_combinations = all_test_combinations[categorical_features]
X_all_combinations_proc = preprocessor.transform(X_all_combinations)

# 5. Fazer a inferência (previsão dos scores)
all_test_combinations['score_new'] = model.predict(X_all_combinations_proc)

# 6. Selecionar as top-3 ofertas para cada cliente
top3_offers_per_client = all_test_combinations.sort_values(
    ['account_id', 'score_new'], ascending=[True, False]
).groupby('account_id').head(3).reset_index(drop=True)

In [6]:
# Unir as top-3 ofertas recomendadas com os dados reais de conversão do test_set
# A junção deve ser feita pelas colunas 'account_id' e 'offer_id'.
top3_with_conversion = pd.merge(
    top3_offers_per_client,
    test_set[['account_id', 'offer_id', 'has_offer_completed']],
    on=['account_id', 'offer_id'],
    how='left'
)

# Retirar nan
top3_with_conversion.dropna(inplace=True)

## Avaliação das Métricas e Impacto de Negócio

Para validar nossas recomendações, calculamos métricas de desempenho comparando-as com uma linha de base (baseline). A taxa de conversão baseline é a taxa de conversão geral das ofertas enviadas no grupo de teste. Em seguida, calculamos a taxa de conversão das nossas recomendações top-3, filtrando apenas as ofertas que existiam no conjunto de teste para uma comparação justa.

A ideia principal é verificar o resultado obtido com o envio real das ofertas com o resultado hipotético se fossem enviadas apenas as Top-3 recomendações do modelo. Essa abordagem, no entanto, pode introduzir vié pois nem todas as ofertas previstas pelo modelo foram efetivamente enviadas.
O ideal seria um teste A/B controlado, no qual um grupo recebe as ofertas segundo a regra de negócio atual e outro recebe as Top-3 ofertas do modelo.

Os resultados indicam que as recomendações do modelo aumentam significativamente a taxa de conversão média (considerando apenas ofertas realmente enviadas), embora a taxa de sucesso (proporção de clientes que usaram pelo menos uma oferta) tenha caído levemente. Além disso, a média de ofertas enviadas no baseline foi de 3 por cliente, enquanto nas Top-3 o número caiu para apenas 1, já que nem todas as ofertas recomendadas estavam no conjunto enviado.

Esse resultado é interessante, pois mesmo com menos ofertas enviadas, é possível obter conversão mais alta e sucesso próximo ao baseline.
Considerando que o envio de ofertas envolve custos, aumentar a precisão enquanto reduz o volume enviado pode ser vantajoso estrategicamente.
Se a empresa adotar essa lógica de priorização, o modelo de recomendação pode trazer contribuições relevantes.

In [11]:
# Taxa de conversão baseline NO GRUPO TESTE (sem ranking, taxa geral no teste)
print(f"Qtd de clientes únicos grupo teste: {len(test_set['account_id'].unique()):,.0f}")
print(f"Qtd de ofertas enviadas grupo teste: {len(test_set['score']):,.0f}")
print(f"Qtd média de ofertas por cliente grupo teste: {test_set[['has_offer_completed','account_id']].groupby('account_id').count()['has_offer_completed'].mean():,.0f}")
baseline_conversion_test = test_set['has_offer_completed'].mean()
base_hit_rate = test_set[['has_offer_completed','account_id']].groupby('account_id').max()['has_offer_completed'].mean()
print(f"Taxa de conversão baseline no GRUPO TESTE: {baseline_conversion_test:.4f}")
print(f"Taxa de sucesso baseline no GRUPO TESTE: {base_hit_rate:.4f}")

print("______________________")
conversion_rate = top3_with_conversion['has_offer_completed'].mean()
print(f"Qtd de clientes com ofertas das Top-3 Recomendações: {(top3_with_conversion['account_id'].nunique())}")
print(f"Qtd de ofertas enviadas Top-3 Recomendações: {len(top3_with_conversion['has_offer_completed']):,.0f}")
print(f"Qtd média de ofertas por cliente Top-3 Recomendações: {top3_with_conversion[['has_offer_completed','account_id']].groupby('account_id').count()['has_offer_completed'].mean():,.0f}")
print(f"Taxa de Conversão Média das Top-3 Recomendações: {conversion_rate:.4f}")
# Agrupar por cliente e verificar se houve alguma conversão (valor máximo = 1)
clients_with_hit = top3_with_conversion.groupby('account_id')['has_offer_completed'].max()
# Contar quantos clientes tiveram pelo menos um acerto
num_clients_with_hit = (clients_with_hit == 1).sum()
# Calcular o Hit Rate
total_clients = len(top3_with_conversion['account_id'].unique())
hit_rate = num_clients_with_hit / total_clients
print(f"Taxa de sucesso das Top-3 Recomendações: {hit_rate:.4f}")

Qtd de clientes únicos grupo teste: 2,183
Qtd de ofertas enviadas grupo teste: 6,531
Qtd média de ofertas por cliente grupo teste: 3
Taxa de conversão baseline no GRUPO TESTE: 0.6288
Taxa de sucesso baseline no GRUPO TESTE: 0.8058
______________________
Qtd de clientes com ofertas das Top-3 Recomendações: 1707
Qtd de ofertas enviadas Top-3 Recomendações: 2,463
Qtd média de ofertas por cliente Top-3 Recomendações: 1
Taxa de Conversão Média das Top-3 Recomendações: 0.7231
Taxa de sucesso das Top-3 Recomendações: 0.7727
